In [1]:
!pip install -q playwright
!playwright install chromium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 19.1 MB/s eta 0:00:00
173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 17.8s173.7 MiB [] 0% 13.0s173.7 MiB [] 0% 8.2s173.7 MiB [] 1% 5.3s173.7 MiB [] 1% 4.2s173.7 MiB [] 2% 3.6s173.7 MiB [] 3% 3.0s173.7 MiB [] 4% 2.7s173.7 MiB [] 5% 2.6s173.7 MiB [] 5% 2.8s173.7 MiB [] 6% 2.6s173.7 MiB [] 7% 2.6s173.7 MiB [] 7% 2.5s173.7 MiB [] 8% 2.4s173.7 MiB [] 9% 2.3s173.7 MiB [] 10% 2.2s173.7 MiB [] 11% 2.2s173.7 MiB [] 13% 2.0s173.7 MiB [] 14% 2.0s173.7 MiB [] 15% 1.9s173.7 MiB [] 16% 1.8s173.7 MiB [] 17% 1.8s173.7 MiB [] 18% 1.7s173.7 MiB [] 19% 1.6s173.7 MiB [] 20% 1.6s173.7 MiB [] 21% 1.6s173.7 MiB [] 22% 1.6s173.7 MiB [] 23% 1.6s173.7 MiB [] 25% 1.5s173.7 MiB [] 26% 1.5s173.7 MiB [] 27% 1.4s173.7 MiB [] 29% 1.3s173.7 MiB [] 30% 1.3s173.7 MiB [] 32% 1.2s173.7 MiB [] 33% 1.2s173.7 MiB [] 34% 1.2s173.7 MiB [] 35% 1.1s173.7 MiB [] 37% 1.1s173.7 MiB [] 39% 1.0s173.7 MiB [] 40% 1.0s173.7 MiB [] 41% 1.0s173.7 MiB [] 42% 1.0s173.7 MiB [] 44% 0.9s173.7 MiB [

In [2]:
!pip -q install nest_asyncio


In [ ]:
# --- BFS Crawl Code ---
import asyncio, json, nest_asyncio
from pathlib import Path
from urllib.parse import urljoin, urlparse
from playwright.async_api import async_playwright

nest_asyncio.apply()

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

START_URL = "https://www.wikipedia.org/"
ALLOWED_DOMAINS = {"wikipedia.org", "wikimedia.org"}
MAX_DEPTH = 2
MAX_PAGES = 20

def is_allowed(url: str) -> bool:
    try:
        host = urlparse(url).hostname or ""
        return any(host.endswith(d) for d in ALLOWED_DOMAINS)
    except Exception:
        return False

def normalize_link(base_url: str, href: str) -> str | None:
    if not href:
        return None
    if href.startswith(("#","mailto:","javascript:")):
        return None
    url = urljoin(base_url, href)
    return url if is_allowed(url) else None

async def bfs_crawl():
    visited = set()
    queue = [(START_URL, 0)]
    manifest = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (AssignmentBot/1.0)")
        page = await context.new_page()

        while queue and len(visited) < MAX_PAGES:
            url, depth = queue.pop(0)
            if url in visited or depth > MAX_DEPTH:
                continue
            visited.add(url)

            status = None
            try:
                resp = await page.goto(url, timeout=30000, wait_until="domcontentloaded")
                status = resp.status if resp else None
            except Exception:
                status = None

            manifest.append({"url": url, "depth": depth, "status": status})

            if depth < MAX_DEPTH and len(visited) < MAX_PAGES:
                try:
                    hrefs = await page.eval_on_selector_all(
                        "a[href]", "els => els.map(e => e.getAttribute('href'))"
                    )
                    for h in hrefs or []:
                        u = normalize_link(url, h)
                        if u and u not in visited and all(u != q[0] for q in queue):
                            queue.append((u, depth + 1))
                            if len(visited) + len(queue) >= MAX_PAGES:
                                break
                except Exception:
                    pass

        await context.close()
        await browser.close()

    return manifest

manifest = await bfs_crawl()   # <-- use await, not asyncio.run

out_path = OUTPUT_DIR / "wikipedia_bfs_manifest.json"
with out_path.open("w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

print(f"Crawled {len(manifest)} pages (BFS).")
for row in manifest[:5]:
    print(f"[depth={row['depth']}] {row['url']} (status={row['status']})")

print(f"\nSaved crawl manifest to {out_path.resolve()}")